# Bank Marketing Dataset


During my journey as a Math student I decided that I should finally try to prepare some data connected projects as I'm also interested in stock market and so on I tried a bit to build some models in this area but it is a bit too challanging for me at the moment so I decided to try prepare a bit simpler project on one of classic dataset. 

## About Dataset
This dataset is related to direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe to a term deposit. It includes features like age, job, marital status, education, and the campaign outcome. This might be very helpful in better targeting marketing campaigns. 

Let's start with loading this dataset and checking with what data we are going to deal 

In [1]:
import pandas as pd
import numpy as np 
df = pd.read_csv("data/bank.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


Columns included in this dataset: <br>
age: Age of the customer. <br>
job: Job role of the customer. \<br>
marital: Marital status of the customer. <br>
education: Education level of the customer. <br>
default: Indicates if the customer has credit in default.<br>
balance: Account balance of the customer.<br>
housing: Indicates if the customer has a housing loan.<br>
loan: Indicates if the customer has a personal loan.<br>
contact: Type of communication contact.<br>
day: Last contact day of the month.<br>
month: Last contact month of the year.<br>
duration: Last contact duration, in seconds.<br>
campaign: Number of contacts performed during this campaign for this client.<br>
pdays: Number of days that passed by after the client was last contacted from a previous campaign (999 means client was not previously contacted).<br>
poutcome: Outcome of the previous marketing campaign.<br>
deposit: Indicates if the client subscribed to a term deposit.

In [8]:
# to start with we will look at summary for numerical variables
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818,2.508421,51.330407,0.832557
std,11.913369,3225.413326,8.420740,347.128386,2.722077,108.758282,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000


In [9]:
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']
categorical_summary = {column: df[column].value_counts() for column in categorical_columns}
categorical_summary

{'job': job
 management       2566
 blue-collar      1944
 technician       1823
 admin.           1334
 services          923
 retired           778
 self-employed     405
 student           360
 unemployed        357
 entrepreneur      328
 housemaid         274
 unknown            70
 Name: count, dtype: int64,
 'marital': marital
 married     6351
 single      3518
 divorced    1293
 Name: count, dtype: int64,
 'education': education
 secondary    5476
 tertiary     3689
 primary      1500
 unknown       497
 Name: count, dtype: int64,
 'default': default
 no     10994
 yes      168
 Name: count, dtype: int64,
 'housing': housing
 no     5881
 yes    5281
 Name: count, dtype: int64,
 'loan': loan
 no     9702
 yes    1460
 Name: count, dtype: int64,
 'contact': contact
 cellular     8042
 unknown      2346
 telephone     774
 Name: count, dtype: int64,
 'month': month
 may    2824
 aug    1519
 jul    1514
 jun    1222
 nov     943
 apr     923
 feb     776
 oct     392
 jan     34

First impressions:
- it seems we're dealing with really fine dataset it's complete(no missing data), well balanced(very similar number of clients in both categories(ones who deposited and ones who didn't))
- variable 'poutcome' might potentially be a bit problematic because over 8000 times it's 'unknown'
- definietly good thing is that we have 0 NaN (missing data)